In [36]:
import pandas as pd
import useful_functions as use
from fuzzywuzzy import fuzz
import pyodbc

In [7]:
cred = pd.read_csv('../../Data/Credentialling/Data_Load.csv')
cred.ME = use.fix_me(cred.ME)
ppd = use.get_ppd()

In [15]:
cred.head()

,entity_id,comm_id,src_cat_code,addr_line2,addr_line1,city_cd,state_cd,zip,ME,ENTITY_ID
0,1234658,26789742,CRED-ADDR,1653 Temple Ave N,P O Drawer 710,Fayette,AL,35555.0,00102000072,1234658
1,1234671,26577992,CRED-ADDR,3719 Dauphin St Frnt,,Mobile,AL,36608.0,00102000081,1234671
2,1234671,26807023,CRED-ADDR,2451 University Hospital Dr,Mastin Bldg - 6th Floor,Mobile,AL,36617.0,00102000081,1234671
3,1234671,26770388,CRED-ADDR,2406 Catalpa Ave,,Pascagoula,MS,39567.0,00102000081,1234671
4,1234671,26740808,CRED-ADDR,2809 Denny Ave,,Pascagoula,MS,39581.0,00102000081,1234671


In [28]:
def is_a_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>75:
        match = True
        print(f'{thing_1} is {thing_2}')
    else:
        match = False
    return(match)

def is_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    else:
        match = False
    return(match)

In [12]:
ppd = ppd[['ME','POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2','POLO_CITY', 'POLO_STATE', 'POLO_ZIP','TELEPHONE_NUMBER']]

In [14]:
all_this = pd.merge(ppd, cred, on='ME')

In [20]:
all_this['cred_add'] = [x.upper() for x in all_this.addr_line2]
all_this['city'] = [x.upper() for x in all_this.city_cd]
all_this['zip'] = [use.fix_zipcode(x) for x in all_this.zip]

In [32]:
all_this = all_this.fillna('None')

In [33]:
adds = 0
zips = 0
phones = 0
for row in all_this.itertuples():
    if row.state_cd == row.POLO_STATE:
        if is_match(str(row.zip), str(row.POLO_ZIP)):
            zips+=1
            if is_a_match(str(row.cred_add), str(row.POLO_MAILING_LINE_2)):
                adds+=1
                if row.TELEPHONE_NUMBER!='None':
                    phones+=1
print(adds)
print(zips)
print(phones)

52 MEDICAL PARK DR E STE 305             is 52 MEDICAL PARK DR E STE 220
2451 UNIVERSITY HOSPITAL DR              is 2451 UNIVERSITY HOSPITAL DR STE F
2250 W SOUTHERN AVE STE 101              is 2250 W SOUTHERN AVE STE 102
1701 W CHARLESTON BLVD STE 110           is 1701 W CHARLESTON BLVD STE 490
5323 HARRY HINES BLVD STOP 7200          is 5323 HARRY HINES BLVD STOP 8859
11782 SW BARNES RD STE 200               is 11782 SW BARNES RD STE 300
15047 LOS GATOS BLVD STE 150             is 15047 LOS GATOS BLVD STE 200
12188 N MERIDIAN ST STE 150              is 12188A N MERIDIAN ST STE 250
633 GOV CARLOS G CAMACHO RD              is 633 GOV CARLOS G CAMACHO RD STE 103
15825 LAGUNA CANYON RD STE 200           is 15785 LAGUNA CANYON RD STE 350
7700 OLD BRANCH AVE STE A101             is 7700 OLD BRANCH AVE STE B101
1203 LANGHORNE NEWTOWN RD STE M10        is 1717 LANGHORNE NEWTOWN RD STE 150
1701 W CHARLESTON BLVD STE 110           is 1707 W CHARLESTON BLVD STE 230
4300 GOLDEN CENTER DR STE E 

3101 SW SAM JACKSON PARK RD              is 3181 SW SAM JACKSON PARK RD
315 MARTIN LUTHER KING JR WAY            is 521 MARTIN LUTHER KING JR WAY
231 W LOCKWOOD AVE STE 201               is 231 W LOCKWOOD AVE STE 202
1501 N CEDAR CREST BLVD STE 100          is 1501 N CEDAR CREST BLVD STE 110
7430 N SHADELAND AVE STE 100             is 7430 N SHADELAND AVE STE 200
901 SAN BERNARDINO RD STE 201            is 1310 SAN BERNARDINO RD STE 201
7625 MAPLE LAWN BLVD STE 110             is 7625 MAPLE LAWN BLVD STE 200
17031 LAKESIDE HILLS PLZ STE 110         is 17030 LAKESIDE HILLS PLZ STE 200
17031 LAKESIDE HILLS PLZ STE 110         is 17001 LAKESIDE HILLS PLZ STE 100
3711 LONG BEACH BLVD STE 101B            is 3711 LONG BEACH BLVD STE 700
11782 SW BARNES RD STE 200               is 11782 SW BARNES RD STE 300
17031 LAKESIDE HILLS PLZ STE 110         is 17030 LAKESIDE HILLS PLZ STE 202
1701 W CHARLESTON BLVD STE 110           is 1707 W CHARLESTON BLVD STE 160
5530 WISCONSIN AVE STE 1620         

3711 LONG BEACH BLVD STE 101B            is 3711 LONG BEACH BLVD STE 700
9301 N CENTRAL EXPY STE 100              is 9301 N CENTRAL EXPY STE 180A
180 NEWPORT CENTER DR STE 157            is 180 NEWPORT CENTER DR STE 158
180 NEWPORT CENTER DR STE 150            is 180 NEWPORT CENTER DR STE 158
9301 N CENTRAL EXPY STE 100              is 9301 N CENTRAL EXPY STE 300
1301 BARBARA JORDAN BLVD STE 100         is 1301 BARBARA JORDAN BLVD STE 400
9301 N CENTRAL EXPY STE 100              is 9301 N CENTRAL EXPY STE 500
9301 N CENTRAL EXPY STE 100              is 9301 N CENTRAL EXPY STE 500
7610 N STEMMONS FWY STE 500              is 7610 N STEMMONS FWY STE 600
1301 BARBARA JORDAN BLVD STE 100         is 1301 BARBARA JORDAN BLVD STE 300
1301 BARBARA JORDAN BLVD STE 100         is 1301 BARBARA JORDAN BLVD STE 200
1240 JESSE JEWELL PKWY SE STE 360        is 1488 JESSE JEWELL PKWY SE STE 100
1120 W LA VETA AVE STE 330               is 1120 W LA VETA AVE STE 300
1501 RIVER POINTE DR STE 200          

In [ ]:
all_this['IQVIA_ME'] = [x[0:10] for x in all_this.ME]

In [37]:
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        B.PHYSICAL_ADDR_1,
        B.PHYSICAL_ADDR_2,
        B.PHYSICAL_CITY,
        B.PHYSICAL_STATE,
        B.PHYSICAL_ZIP,
        P.ME
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,ME
0,2519493400,1613 N MCKENZIE ST,None,FOLEY,AL,36535-2247,0010200001
1,7275885200,201 14TH ST SW,None,LARGO,FL,33770-3133,0010200001
2,8282131111,428 BILTMORE AVE,None,ASHEVILLE,NC,28801-4502,0010200003
3,8282131111,509 BILTMORE AVE,None,ASHEVILLE,NC,28801-4601,0010200003
4,8282544337,500 CENTREPARK DR,None,ASHEVILLE,NC,28805-1262,0010200003


In [ ]:
iqvia = iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [use.fix_phone(x) if x !='None' else x for x in iqvia['PHONE']]

In [ ]:
#data.gov
gov = pd.read_csv('../../Data/DataGov/All_Data_210826.csv', low_memory=False)
gov = gov.fillna('None')
gov['ME'] = use.fix_me(gov.ME)
gov = gov[gov.ME.isin(older_polos.ME)]
gov['GOV_PHONE'] = [use.fix_phone(x) if x !='None' else x for x in gov['phone']]
gov = gov.rename(columns={
    'zip':'ZIP_GOV',
    'st':'STATE_GOV',
    'cty':'CITY_GOV',
    'adr_ln_2': 'ADDRESS_2_GOV',
    'adr_ln_1': 'ADDRESS_1_GOV'
})